In [1]:
import pandas as pd
import os
from IPython.display import Audio
from tqdm import tqdm

In [ ]:
# Создание dataframe с аудиозаписями из датасета RJD.

df = pd.DataFrame() 
data_path = './ржд 1/ESC_DATASET_v1.2/'
annotation_path = os.path.join(data_path, 'annotation')
for d in os.listdir(annotation_path):
    if d.endswith('.json'):
        anno_df = pd.read_json(os.path.join(annotation_path, d))
        dataset_name = d.split('.')[0]
        anno_df['audio_filepath'] = anno_df['audio_filepath'].apply(lambda x: os.path.join(data_path, dataset_name, x))
        anno_df['dataset'] = dataset_name
        anno_df = anno_df.dropna()
        anno_df = anno_df.reset_index(drop=True)
        df = pd.concat([df, anno_df])

df['sample_rate'] = df['audio_filepath'].apply(lambda x: sf.read(x)[1])

print(df[['audio_filepath', 'sample_rate']].head())

audio, sample_rate = sf.read(df.iloc[1].audio_filepath)
print(f"Пример аудио: форма = {audio.shape}, сэмплрейт = {sample_rate}")
df.to_csv('df.csv', index=False)

In [3]:
### Создание тестового и тренировочного наборов
### В тестовом наборе по 5 случайных аудиозаписей для каждого класса из целевого датасета LUGA.

df = pd.read_csv('./df.csv')

luga_df = df[df['dataset'] == 'luga']
min_samples = 5

test_df = luga_df.groupby('label').apply(lambda x: x.sample(min_samples)).reset_index(drop=True)
train_df = df[~df.index.isin(test_df.index)]

print(f"Размер тренировочного набора: {len(train_df)}")
print(f"Размер валидационного набора: {len(test_df)}")


Размер тренировочного набора: 6310
Размер валидационного набора: 115


/tmp/ipykernel_2275428/416793800.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_df = luga_df.groupby('label').apply(lambda x: x.sample(min_samples)).reset_index(drop=True)


In [3]:
### Инициализация и выбор модели

import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import librosa
import time
import psutil
import os
import numpy as np

# Загрузка модели и процессора
start_time = time.time()
model_name = "bond005/wav2vec2-base-ru"
# model_name = "philschmid/tiny-random-wav2vec2"
# model_name = "emre/wav2vec2-xls-r-300m-Russian-small"
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name)

# Убедитесь, что модель работает на CPU
model = model.to("cpu")
load_time = time.time() - start_time
print(f"Время загрузки модели: {load_time:.2f} секунд")

# Функция для транскрибации аудио
def transcribe_audio(audio):
    start_time = time.time()
    

    
    # Подготовка входных данных
    input_values = processor(audio, sampling_rate=16000, return_tensors="pt").input_values
    
    # Инференс
    with torch.no_grad():
        logits = model(input_values).logits

    # Декодирование результатов
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)[0]
    
    process_time = time.time() - start_time
    
    # Получение информации о использовании ресурсов
    process = psutil.Process(os.getpid())
    memory_info = process.memory_info()
    
    print(f"Время обработки: {process_time:.2f} секунд")
    print(f"Длительность аудиозаписи: {audio_duration:.2f} секунд")
    print(f"Отношение скорости работы к записи: {process_time / audio_duration:.2f}")
    print(f"Использование памяти: {memory_info.rss / 1024 / 1024:.2f} МБ")
    print(f"Использование CPU: {psutil.cpu_percent()}%")
    
    return transcription

/opt/personal_data/vyaches/hacks/hacks_ai/2024/spb/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Время загрузки модели: 1.80 секунд


In [9]:
#### Тестирование модели

model.to('cpu')

n = 550
audio_path = test_df.iloc[n].audio_filepath

# Загрузка аудио файла и понижение сэмпл-рейта до 16 кГц
audio_data, rate = librosa.load(audio_path, sr=16000)
#audio_data, rate = process_audio(audio_data, rate)    
# Получение длительности аудиозаписи
audio_duration = librosa.get_duration(y=audio_data, sr=rate)

result = transcribe_audio(audio_data)
print("\n")
print("Транскрипция:", result)
print("Оригинал:", df.iloc[n].text)
display(Audio(audio_path))

Время обработки: 0.25 секунд
Длительность аудиозаписи: 5.00 секунд
Отношение скорости работы к записи: 0.05
Использование памяти: 4265.39 МБ
Использование CPU: 5.2%


Транскрипция: протянуть на шестнадцать вагонов
Оригинал: протянуть на шестнадцать вагонов


In [4]:
#### Файтюнинг модели

import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2CTCTokenizer
from datasets import Dataset
import librosa
import numpy as np
from dataclasses import dataclass
from typing import Dict, List, Union
import pandas as pd
from jiwer import wer
import os
from datetime import datetime

# Функция для загрузки аудио
def load_audio(file_path):
    audio, _ = librosa.load(file_path, sr=16000)
    return audio

# Создание датасета
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Загрузка процессора и модели
model_name = "bond005/wav2vec2-base-ru"
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name)

# Подготовка данных
def prepare_dataset(batch):
    audio = load_audio(batch["audio_filepath"])
    batch["input_values"] = processor(audio, sampling_rate=16000).input_values[0]
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

train_dataset = train_dataset.map(prepare_dataset, remove_columns=train_dataset.column_names)
test_dataset = test_dataset.map(prepare_dataset, remove_columns=test_dataset.column_names)

@dataclass
class DataCollatorCTCWithPadding:
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels
        return batch

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

# Настройка обучения
from transformers import Trainer, TrainingArguments

# Функция для вычисления WER
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)
    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id
    pred_str = processor.batch_decode(pred_ids)
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)
    wer_metric = wer(label_str, pred_str)
    return {"wer": wer_metric}

# Создание уникальной директории для логов
current_time = datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = os.path.join('./logs', current_time)
os.makedirs(log_dir, exist_ok=True)

training_args = TrainingArguments(
    output_dir=f"./wav2vec2-ru-finetuned-{current_time}",
    group_by_length=True,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    num_train_epochs=20,
    fp16=True,
    logging_steps=20,
    learning_rate=1e-4,
    warmup_steps=100,
    eval_steps=200,
    save_total_limit=2,
    logging_dir=log_dir,  # директория для логов TensorBoard
    report_to=["tensorboard"],  # включение отчетности TensorBoard
)

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=processor.feature_extractor,
    compute_metrics=compute_metrics
)

# Запуск обучения
trainer.train()

# Сохранение модели
model.save_pretrained(f"./wav2vec2-ru-finetuned-final-{current_time}")
processor.save_pretrained(f"./wav2vec2-ru-finetuned-final-{current_time}")

Map:   0%|          | 0/6310 [00:00<?, ? examples/s]/opt/personal_data/vyaches/hacks/hacks_ai/2024/spb/.venv/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 115/115 [00:00<00:00, 765.29 examples/s]
/opt/personal_data/vyaches/hacks/hacks_ai/2024/spb/.venv/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/personal_data/vyaches/hacks/hacks_ai/2024/spb/.venv/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will

Step,Training Loss,Validation Loss,Wer
200,0.294400,0.163174,0.146617
400,0.216900,0.042164,0.075188
600,0.104200,0.032850,0.030075
800,0.057800,0.032011,0.048872
1000,0.038000,0.034871,0.048872


/opt/personal_data/vyaches/hacks/hacks_ai/2024/spb/.venv/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/opt/personal_data/vyaches/hacks/hacks_ai/2024/spb/.venv/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


KeyboardInterrupt: 

In [6]:
import tensorboard

In [6]:
# Конвертация модели wav2vec2 в ONNX формат

import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import onnx
import onnxruntime

# Подготовка примера входных данных
dummy_input = torch.randn(1, 16000)

model.to('cpu')

# Экспорт модели в ONNX формат
torch.onnx.export(model, 
                  dummy_input, 
                  "wav2vec2_russian_train_2.onnx", 
                  input_names=['input'], 
                  output_names=['output'], 
                  dynamic_axes={'input': {0: 'batch_size', 1: 'sequence'}, 
                                'output': {0: 'batch_size', 1: 'sequence_length'}},
                  opset_version=14)

print("Модель успешно конвертирована в ONNX формат")

# Проверка конвертированной модели
onnx_model = onnx.load("wav2vec2_russian.onnx")
onnx.checker.check_model(onnx_model)

print("ONNX модель проверена и корректна")

# Тестирование инференса с использованием ONNX Runtime
ort_session = onnxruntime.InferenceSession("wav2vec2_russian_train_2.onnx")

# Подготовка входных данных для теста
input_data = torch.randn(1, 16000)
ort_inputs = {ort_session.get_inputs()[0].name: input_data.numpy()}

# Выполнение инференса
ort_outputs = ort_session.run(None, ort_inputs)

print("Инференс с использованием ONNX Runtime выполнен успешно")


/opt/personal_data/vyaches/hacks/hacks_ai/2024/spb/.venv/lib/python3.10/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:871: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if attn_output.size() != (bsz, self.num_heads, tgt_len, self.head_dim):


Модель успешно конвертирована в ONNX формат
ONNX модель проверена и корректна
Инференс с использованием ONNX Runtime выполнен успешно


In [5]:
#### Инференс модели ONNX

import torch
import librosa
import time
import psutil
import os
import numpy as np

import onnxruntime as ort
from transformers import Wav2Vec2Processor

sess_options = ort.SessionOptions()
sess_options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL
sess_options.intra_op_num_threads = 8  # Оптимальное число потоков
sess_options.inter_op_num_threads = 8
# sess_options.set_memory_limit(1024)  # Ограничение памяти в 1 ГБ (1024 МБ)

# Загрузка ONNX модели
start_time = time.time()
ort_session = ort.InferenceSession("wav2vec2_russian_train_2.onnx", sess_options)
load_time = time.time() - start_time
print(f"Время загрузки модели: {load_time:.2f} секунд")

# Загрузка процессора
model_name = "bond005/wav2vec2-base-ru"
processor = Wav2Vec2Processor.from_pretrained(model_name)

# Функция для транскрибации аудио
def transcribe_audio(audio, logs=False):
    start_time = time.time()
    
    # Подготовка входных данных
    input_values = processor(audio, sampling_rate=16000, return_tensors="pt").input_values

    # Получение длительности аудиозаписи
    audio_duration = len(audio) / 16000  # Предполагаем частоту дискретизации 16000 Гц
    
    # Инференс
    ort_inputs = {ort_session.get_inputs()[0].name: input_values.numpy()}
    ort_outputs = ort_session.run(None, ort_inputs)
    logits = torch.tensor(ort_outputs[0])

    # Декодирование результатов
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)[0]
    
    process_time = time.time() - start_time
    
    # Получение информации о использовании ресурсов
    process = psutil.Process(os.getpid())
    memory_info = process.memory_info()
    
    if logs:
        print(f"Время обработки(latency): {process_time * 1000:.2f} мс")
        print(f"Длительность аудио: {audio_duration:.2f} секунд")
        print(f"Отношение скорости работы к записи: {process_time / audio_duration:.2f}")
        print(f"Использование памяти(memory): {memory_info.rss / 1024 / 1024:.2f} МБ")
        print(f"Использование CPU: {psutil.cpu_percent()}%")
    
    return transcription


Время загрузки модели: 0.85 секунд


In [6]:
### Тестирование модели

n = 34
audio_path = train_df.iloc[n].audio_filepath
# audio_path = 'audio2.wav'

# Загрузка аудио файла и понижение сэмпл-рейта до 16 кГц
audio_data, rate = librosa.load(audio_path, sr=16000)
#audio_data, rate = process_audio(audio_data, rate)    

result = transcribe_audio(audio_data, logs=True)
print("\n")
print("Транскрипция:", result)
print("Оригинал:", train_df.iloc[n].text)
display(Audio(audio_path))

Время обработки(latency): 227.12 мс
Длительность аудио: 5.00 секунд
Отношение скорости работы к записи: 0.05
Использование памяти(memory): 1431.36 МБ
Использование CPU: 1.0%


Транскрипция: подтверждение
Оригинал: подтверждение


In [11]:
### Обработка аудиозаписей из тестового набора

# Создаем список для хранения результатов
results = []

# Проходим по всем строкам
for index, row in tqdm(test_df.iterrows()):
    audio_path = row.audio_filepath
    
    # Загрузка аудио файла
    audio_data, rate = librosa.load(audio_path, sr=16000)
    
    # Транскрибирование аудио
    transcription = transcribe_audio(audio_data, logs=False)
    
    # Добавляем результат в список
    results.append({
        'id': row.id,
        'original_text': row.text,
        'transcription': transcription
    })

# Создаем DataFrame из результатов
results_df = pd.DataFrame(results)


115it [00:19,  5.87it/s]


In [14]:
### Расчет WER для тестового набора


# Импортируем необходимую библиотеку для расчета WER
from jiwer import wer

# Рассчитываем WER для каждой пары оригинального текста и транскрипции
results_df['wer'] = results_df.apply(lambda row: wer(row['original_text'], row['transcription']), axis=1)

# Вычисляем средний WER по всем образцам
average_wer = results_df['wer'].mean()

print(f"Средний WER: {average_wer:.4f}")

# Выводим DataFrame с добавленным столбцом WER
results_df[results_df['original_text'] != results_df['transcription']]

results_df = pd.merge(test_df, results_df, on='id')[['id','original_text', 'transcription', 'wer', 'label','attribute']]

Средний WER: 0.0196


In [15]:
#### Инициализация словарей для распознавания команд

commands_dict = {'назад с башмака': 17,
 'прекратить зарядку тормозной магистрали': 20,
#  'осадить на двадцать восемь вагонов': 4,
 'зарядка тормозной магистрали': 6,
 'вышел из межвагонного пространства': 7,
#  'протянуть на шестнадцать вагонов': 10,
 'вперед на башмак': 15,
 'вперед с башмака': 19,
 'остановка': 14,
 'отцепка': 11,
 'отказ': 0,
 'растянуть автосцепки': 9,
 'продолжаем роспуск': 8,
 'захожу в межвагонное пространство': 13,
 'тормозить': 21,
 'отмена': 1,
 'тише': 18,
 'отпустить': 22,
 'начать осаживание': 3,
 'сжать автосцепки': 16,
 'продолжаем осаживание': 5,
 'назад на башмак': 12,
 'подтверждение': 2}

count_commands_dict = {'протянуть': 10,
 'осадить': 4,}
 
# Словарь для чисел
numbers_dict = {
    'один': { "count": 1, "text":"один вагон" } , 'два': {"count":2, "text":"два вагона"}, 'три': {"count":3, "text":"три вагона"}, 'четыре': {"count":4, "text":"четыре вагона"}, 'пять': {"count":5, "text":"пять вагонов"},
    'шесть': {"count":6, "text":"шесть вагонов"}, 'семь': {"count":7, "text":"семь вагонов"}, 'восемь': {"count":8, "text":"восемь вагонов"}, 'девять': {"count":9, "text":"девять вагонов"}, 'десять': {"count":10, "text":"десять вагонов"},
    'одиннадцать': {"count":11, "text":"одиннадцать вагонов"}, 'двенадцать': {"count":12, "text":"двенадцать вагонов"}, 'тринадцать': {"count":13, "text":"тринадцать вагонов"}, 'четырнадцать': {"count":14, "text":"четырнадцать вагонов"},
    'пятнадцать': {"count":15, "text":"пятнадцать вагонов"}, 'шестнадцать': {"count":16, "text":"шестнадцать вагонов"}, 'семнадцать': {"count":17, "text":"семнадцать вагонов"}, 'восемнадцать': {"count":18, "text":"восемнадцать вагонов"},
    'девятнадцать': {"count":19, "text":"девятнадцать вагонов"}, 'двадцать': {"count":20, "text":"двадцать"}, 'тридцать': {"count":30, "text":"тридцать"}, 'сорок': {"count":40, "text":"сорок"},
    'девяносто': {"count":90, "text":"девяносто"}, 'сто': {"count":100, "text":"сто"}, 'двести': {"count":200, "text":"двести"}, 'триста': {"count":300, "text":"триста"},
    'четыреста': {"count":400, "text":"четыреста"}, 'пятьсот': {"count":500, "text":"пятьсот"}, 'шестьсот': {"count":600, "text":"шестьсот"}, 'семьсот': {"count":700, "text":"семьсот"},
    'восемьсот': {"count":800, "text":"восемьсот"}, 'девятьсот': {"count":900, "text":"девятьсот"}, 'тысяча': {"count":1000, "text":"тысяча"}
}


In [16]:
#### Алгоритм распознавания команд на основе FuzzyWuzzy + расчет итоговых мерик на тестовом наборе.

# Импортируем необходимые библиотеки
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import time
import numpy as np
import re

# Вычисляем средний WER до работы алгоритма
average_wer_before = results_df['wer'].mean()
print(f"Средний WER до работы алгоритма: {average_wer_before:.4f}")

# Функция для поиска ближайшей команды
def find_closest_command(transcription, commands, count_commands, numbers, threshold=80):
    # Поиск ближайшей команды по всей фразе8
    closest_command = process.extractOne(transcription, commands.keys(), scorer=fuzz.ratio)
    command = closest_command[0]
    if closest_command[1] >= threshold:
        return command, commands[command], -1
    
    words = transcription.lower().split()
    for word in words:
        closest_word = process.extractOne(word, count_commands.keys(), scorer=fuzz.ratio)
        if closest_word[1] >= 95:
            command = closest_word[0]
            # Поиск числа вагонов
            number_words = []
            total_count = 0
            for word in transcription.lower().split():
                closest_number = process.extractOne(word, numbers.keys(), scorer=fuzz.ratio)
                if closest_number[1] >= threshold:
                    number_key = closest_number[0]
                    number_value = numbers[number_key]
                    number_words.append(number_value['text'])
                    total_count += number_value['count']
                elif word.isdigit():
                    count = int(word)
                    total_count += count
                    number_words.append(f"{count} вагонов")
            
            if number_words:
                return f"{closest_word[0]} на {' '.join(number_words)}", count_commands[command], total_count
            else:
                return closest_word[0], count_commands[command], -1
    return "Команда не распознана", -1, -1

# Инициализируем список для хранения времени выполнения каждой итерации
execution_times = []

# Выполняем поиск ближайшей команды для каждой транскрипции
for _, row in results_df.iterrows():
    start_time = time.time()
    closest_command, label, count = find_closest_command(row['transcription'], commands_dict, count_commands_dict, numbers_dict)
    end_time = time.time()
    execution_times.append(end_time - start_time)
    
    # Сохраняем результат
    results_df.at[_, 'closest_command'] = closest_command
    results_df.at[_, 'predicted_label'] = label
    results_df.at[_, 'predicted_attribute'] = count

# Вычисляем среднее время выполнения поиска для одной команды
average_execution_time = np.mean(execution_times)

# Рассчитываем WER для найденных команд
results_df['command_wer'] = results_df.apply(lambda row: wer(row['original_text'], row['closest_command']), axis=1)

# Вычисляем средний WER для найденных команд
average_command_wer = results_df['command_wer'].mean()

print(f"Средний WER после поиска ближайшей команды: {average_command_wer:.4f}")
print(f"Среднее время выполнения поиска для одной команды: {average_execution_time:.6f} секунд")

# Выводим обновленный DataFrame
results_df[results_df['original_text'] != results_df['closest_command']]

# Импортируем необходимые библиотеки
from sklearn.metrics import f1_score

# Функция для вычисления F1-score для атрибутов
def attribute_f1_score(y_true, y_pred):
    # Преобразуем -1 в None для корректного сравнения
    y_true = [None if x == -1 else x for x in y_true]
    y_pred = [None if x == -1 else x for x in y_pred]
    # Считаем True Positives, False Positives и False Negatives
    tp = sum(1 for t, p in zip(y_true, y_pred) if t == p and t is not None)
    fp = sum(1 for t, p in zip(y_true, y_pred) if t != p and p is not None)
    fn = sum(1 for t, p in zip(y_true, y_pred) if t != p and t is not None)
    # Вычисляем precision и recall
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    # Вычисляем F1-score
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f1

# Вычисляем F1-score для меток (label)
label_f1 = f1_score(results_df['label'], results_df['predicted_label'], average='weighted')

# Вычисляем F1-score для атрибутов (attribute)
attribute_f1 = attribute_f1_score(results_df['attribute'], results_df['predicted_attribute'])

print(f"F1-score для меток (label): {label_f1:.4f}")
print(f"F1-score для атрибутов (attribute): {attribute_f1:.4f}")

# Функция для вычисления метрики MQ
def calculate_mq(average_command_wer, label_f1, attribute_f1):
    mq = 0.25 * (1 - average_command_wer) + 0.75 * (label_f1 + attribute_f1) / 2
    return mq

# Вычисляем метрику MQ
mq_score = calculate_mq(average_command_wer, label_f1, attribute_f1)

print(f"Метрика MQ: {mq_score:.4f}")



Средний WER до работы алгоритма: 0.0196
Средний WER после поиска ближайшей команды: 0.0017
Среднее время выполнения поиска для одной команды: 0.001098 секунд
F1-score для меток (label): 1.0000
F1-score для атрибутов (attribute): 1.0000
Метрика MQ: 0.9996
